In [226]:
# Cargar librerías
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
import mlflow
import mlflow.sklearn


In [227]:
if mlflow.active_run():
    mlflow.end_run()

In [228]:
# Leer datos
df = pd.read_csv("data/diabetes.csv")



In [229]:
# identifica las columnas que tiene valores 0
(df==0).sum()

Pregnancies                 111
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                     500
dtype: int64

In [230]:
# Reemplazar ceros en columnas específicas por NaN
cols_to_clean = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin']
# No se consideran algunas, por ejemplo Pregnancies (por qué si es posible que se tengan cero embarazos)

# Reemplazar ceros por NaN para poder tratarlos como datos faltantes
for col in cols_to_clean:
    df[col] = df[col].replace(0, np.nan)

# Reemplazar NaN con la moda (valor más frecuente) de cada columna
for col in cols_to_clean:
    moda = df[col].mode()[0]# buscar el método más común para determinar la moda de una columna
    df[col] = df[col].fillna(moda)
print (df)
print (moda)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6    148.0           72.0           35.0    105.0  33.6   
1              1     85.0           66.0           29.0    105.0  26.6   
2              8    183.0           64.0           32.0    105.0  23.3   
3              1     89.0           66.0           23.0     94.0  28.1   
4              0    137.0           40.0           35.0    168.0  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10    101.0           76.0           48.0    180.0  32.9   
764            2    122.0           70.0           27.0    105.0  36.8   
765            5    121.0           72.0           23.0    112.0  26.2   
766            1    126.0           60.0           32.0    105.0  30.1   
767            1     93.0           70.0           31.0    105.0  30.4   

     DiabetesPedigreeFunction  Age  Outcome  
0                       0.627   50        1  
1                  

In [231]:
# Separar datos
X = df.drop("Outcome", axis=1)
y = df["Outcome"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [232]:
# agregar el traking uri de mlflow
mlflow.set_tracking_uri("http://127.0.0.1:9090")

# agregar set_experiment con nombre: ClasificadorDemoDiabetes
mlflow.set_experiment(experiment_name="ClasificadorDemoDiabetes")

<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1747445244365, experiment_id='0', last_update_time=1747445753960, lifecycle_stage='active', name='ClasificadorDemoDiabetes', tags={}>

In [233]:
# Entrenamiento y registro con MLflow
C = 0.01
max_iter = 10
# usar los parámetros del archivo logreg_variaciones_educativas.csv
df_par= pd.read_csv('data/logreg_variaciones_educativas.csv')
print(df_par)
df_par.dtypes
# para los parámtros C, max_iter, solver y penalty de LogisticRegresion
# se debe genear un run por cada fila del archivo, usando sus parámetros
# Analizar que usar
# agregar la línea del start_run

for _, row in df_par.iterrows():
    with mlflow.start_run(run_name="CLASSn"):  
        C = row['logreg_C']
        max_iter = row['logreg_max_iter']
        solver = row['solver']
        penalty = row['penalty']        
    # El pipeline usar StandarScaler para que todos los valore numéricos estén en la misma escala
    # no cambiar dicha línea
    # LogisticRegresion deben variar sus parámetros en cada iteración.
        pipeline = Pipeline([
            ("scaler", StandardScaler()),
            ("clf", LogisticRegression(
                C=C,
                max_iter=max_iter,
                solver=solver,
                penalty=penalty
            ))
        ])
    
    # Entrenar y evaluar
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)

    # recuerde que log_param y log_metric debe ir guardando los valores
    # dependiendo del run correspondiente de cada fila
    mlflow.log_param("logreg_C", C)
    mlflow.log_param("logreg_max_iter", max_iter)
    mlflow.log_param("solver", solver)
    mlflow.log_param("penalty", penalty)
    
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("precision", prec)
    
    # Guardar el pipeline completo
    #mlflow.sklearn.log_model(pipeline, "modelo_pipeline")
    model_info = mlflow.sklearn.log_model(
        sk_model=pipeline,
        artifact_path="modelo_pipeline",
        registered_model_name="modelo_pipeline"    
    )
    print(" Modelo registrado en MLflow")
    print(f"\nModelo registrado - Parámetros: C={C}, max_iter={max_iter}, solver={solver}, penalty={penalty}")
    print(f"Accuracy: {acc:.4f} | Precision: {prec:.4f}")


        run_id  logreg_C  logreg_max_iter     solver penalty
0  practica_01      0.01              400  liblinear      l2
1  practica_02      1.12              300      lbfgs      l2
2  practica_03      2.23              500  liblinear      l2
3  practica_04      3.34              200  liblinear      l2
4  practica_05      4.45              300  liblinear      l2
5  practica_06      5.56              500      lbfgs      l2
6  practica_07      6.67              300      lbfgs      l2
7  practica_08      7.78              500      lbfgs      l2
8  practica_09      8.89              400      lbfgs      l2
9  practica_10     10.00              200      lbfgs      l2


C:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
C:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'modelo_pipeline' already exists. Creating a new version of this model...
2025/05/16 23:22:07 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: modelo_pipeline, version 3


 Modelo registrado en MLflow

Modelo registrado - Parámetros: C=0.01, max_iter=400, solver=liblinear, penalty=l2
Accuracy: 0.7208 | Precision: 0.6034


Created version '3' of model 'modelo_pipeline'.


Exception: Run with UUID 77f38fe9388c4391b6008b3fe0438ee5 is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True